In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('data/data.csv').set_index(['equity','date'])

In [3]:
df_prices = df['PX_LAST'].unstack(level=0).replace(0, np.nan)
df_log_rtn = df_prices.apply(lambda x: np.log(x / x.shift(1))).dropna(how='all')

In [4]:
benchmark_rtn = df_log_rtn.mean(skipna=True, axis='columns')

In [6]:
def select_best_equities_from_factor(df_full, df_rnt, factor_selected, num_equities):
    df_factor_values = df_full[factor_selected].unstack(level=0).iloc[1:]
    equities =  df_factor_values.apply(
        lambda fact_val: fact_val.loc[~df_rnt.loc[fact_val.name, fact_val.index].isna()
        ].sort_values(ascending=False).head(num_equities).index.values,
        axis='columns', result_type='expand')
    equities.columns = [str(i) + "_name" for i in range(len(equities.columns))]
    return equities

def compute_return_from_equities(df_rtn, equities):
    equities_rtn = equities.apply(lambda row: df_rtn.loc[row.name, row.values].values, axis='columns', result_type='expand')
    equities_rtn.columns = [str(i) + "_rtn" for i in range(len(equities_rtn.columns))]
    equities_rtn.columns.name = 'equity_rtn'
    return equities_rtn

def compute_univariate_strategy(df_full, df_rtn, factor_selected, num_equities, market_rtn):
    equities_names = select_best_equities_from_factor(df_full, df_rtn, factor_selected, num_equities)
    equities_rtn = compute_return_from_equities(df_rtn, equities_names)

    equities = equities_names.merge(equities_rtn, left_index=True, right_index=True)
    equities = equities.iloc[:, np.concatenate([[i, i + num_equities] for i in range(num_equities)])]
    equities.columns = pd.MultiIndex.from_product([range(num_equities), ['name', 'rtn']])
    equities.columns.names = ['equity_pos', 'info']

    ris = equities.apply(lambda row: np.average(row.loc[:, "rtn"].values), axis='columns').to_frame(name='strategy_rtn')
    ris['strategy_alpha'] = ris['strategy_rtn'] - market_rtn
    info_ratio = ris['strategy_alpha'].mean() / ris['strategy_alpha'].std()
    return equities, ris, info_ratio

In [7]:
strategy_eq, strategy_ris, strategy_info_rate = compute_univariate_strategy(df, df_log_rtn, 'EBITDA_MARGIN', 10, benchmark_rtn)

In [8]:
strategy_info_rate

0.15626892318432908

In [9]:
strategy_eq

equity_pos       0                 1                  2                 3  \
info          name       rtn    name       rtn     name       rtn    name   
date                                                                        
2003-02-28  RCH GR -0.033907  SIS IM  0.108965    AT IM  0.001857  TIT IM   
2003-03-31  RCH GR  0.000000  SIS IM  0.026688    AT IM  0.014626  TIT IM   
2003-04-30  RCH GR  0.000000  SIS IM  0.035347    AT IM  0.053093  TIT IM   
2003-05-30  RCH GR  0.082688  SIS IM  0.113437    AT IM  0.050125  TIT IM   
2003-06-30  RCH GR  0.015751  SIS IM  0.124420    AT IM  0.020261  TIT IM   
...            ...       ...     ...       ...      ...       ...     ...   
2011-11-30  TUB BB -0.049611  EUR FP -0.170958  HENX BB -0.142322  EIS GR   
2011-12-30  TUB BB  0.045912  EUR FP -0.038001  HENX BB  0.040209  EIS GR   
2012-01-31  TUB BB -0.045912  EUR FP  0.131769  HENX BB  0.046543  EIS GR   
2012-02-29  TUB BB -0.039566  EUR FP -0.012423  HENX BB  0.041964  EIS GR   
2012-03-30  TUB BB  0.118679  EUR FP -0.041473  HENX BB -0.021928  EIS GR   

equity_pos                  4                 5                  6            \
info             rtn     name       rtn    name       rtn     name       rtn   
date                                                                           
2003-02-28 -0.065044   TEF SM  0.002220  SAN FP  0.019742   KPN NA -0.059955   
2003-03-31  0.058836   TEF SM -0.029151  KPN NA -0.044895  DYK3 GR  0.029117   
2003-04-30  0.073684   TEF SM  0.145280  KPN NA  0.013514  DYK3 GR  0.447963   
2003-05-30  0.053519   TEF SM -0.037006  KPN NA -0.005046  DYK3 GR  0.113228   
2003-06-30  0.020278   TEF SM  0.056980  KPN NA  0.039675  DYK3 GR  0.261201   
...              ...      ...       ...     ...       ...      ...       ...   
2011-11-30 -0.114880   RCH GR -0.040274  MUK GR  0.019315   ETL FP -0.034422   
2011-12-30  0.109049  O1BC GR -0.200626  RCH GR  0.231329   MUK GR -0.014737   
2012-01-31 -0.005865  O1BC GR  0.124446  RCH GR -0.139762   MUK GR -0.021548   
2012-02-29  0.019418  O1BC GR  0.049190  RCH GR  0.093035   MUK GR -0.012919   
2012-03-30 -0.063513  O1BC GR  0.118906  RCH GR -0.229427   MUK GR  0.024135   

equity_pos        7                   8                   9            
info           name       rtn      name       rtn      name       rtn  
date                                                                   
2003-02-28   GEM IM  0.071534    VPK NA -0.086626  MLHMC FP -0.016807  
2003-03-31   SAN FP -0.073364    GEM IM  0.012360    VPK NA -0.047784  
2003-04-30   SAN FP  0.147717    GEM IM  0.075772    VPK NA  0.017825  
2003-05-30   SAN FP  0.016698    GEM IM -0.037410    VPK NA  0.104284  
2003-06-30   SAN FP -0.063619    FTE FP  0.016522    ELG GR -0.055489  
...             ...       ...       ...       ...       ...       ...  
2011-11-30   MAU FP -0.021115  SMTPC FP  0.005797   SESG FP -0.011143  
2011-12-30  FLUX BB  0.013859    ETL FP  0.044422    MAU FP -0.017566  
2012-01-31  FLUX BB -0.005279    ETL FP -0.060676    MAU FP  0.077292  
2012-02-29  FLUX BB  0.008824    ETL FP -0.013661    MAU FP  0.113708  
2012-03-30  FLUX BB  0.133619    ETL FP -0.009693    MAU FP -0.076624  

[110 rows x 20 columns]